# Drinking Task Pipeline


Diese Pipeline führt eine Klassifikation von Trinkbewegungen durch.

Die Elemente, welche angepasst werden können, um Ansätze auszuprobieren, wurden mit ✏️ gekennzeichnet.

#### Geeignete Parameter (Beispiel):

**3.2 Ansatz auswählen**
- Ansatz 1: Absolute Datenpunkte mit geschnittenen Videos
- n_frames: **38**
    
**4.3 Modell anwenden**
- n_neurons: **64**
- RNN-Modell

# 1. Import Dependecies

In [315]:
import pandas as pd
import numpy as np

from matplotlib import pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score

from tensorflow import keras
from tensorflow.keras import layers

from sklearn.model_selection import RandomizedSearchCV

import math

# 2. Landmarks extrahieren

In [316]:
def extract_landmarks_normal():
    %run extract_landmarks.py --video-dir data/video/normal --output-file data/landmarks_normal.csv

In [317]:
def extract_landmarks_compensation():
    %run extract_landmarks.py --video-dir data/video/compensation --output-file data/landmarks_compensation.csv

In [318]:
# Falls die Landmarks neu extrahiert werden sollen, folgende Befehle auskommentieren: ✏️
# extract_landmarks_normal()
# extract_landmarks_compensation()

# 3. Data Pre-Processing

In [319]:
# Landmarks einlesen
data_normal_original = pd.read_csv('data/landmarks_normal.csv')
data_compensation_original = pd.read_csv('data/landmarks_compensation.csv')

# Augmentierte Datenpunkte für Datenanalyse einlesen
data_normal_augmented = pd.read_csv('../../AvatarDataAugmentation/Data/KeyPointsBereinigtNichtKompensiert_03.csv')
data_compensation_augmented = pd.read_csv('../../AvatarDataAugmentation/Data/KeyPointsBereinigtKompensiert_01.csv')
data_compensation_augmented2 = pd.read_csv('../../AvatarDataAugmentation/Data/KeyPointsBereinigtKompensiert_02.csv')

# Zusätzliche aus Unity generierte Datenpuntke einlesen
data_normal_augmented_unity = pd.read_csv('../../AvatarDataAugmentation/Data/KeypointsExportKompensiert.csv')
data_compensation_augmented_unity = pd.read_csv('../../AvatarDataAugmentation/Data/KeypointsExportKompensiert.csv')

data_normal = pd.concat([data_normal_original, data_normal_augmented, data_normal_augmented_unity], ignore_index=True) 
data_compensation = pd.concat([data_compensation_original, data_compensation_augmented2, data_compensation_augmented_unity], ignore_index=True) 

# Label setzen (Kompensation = 1, keine Kompensation = 0)
""" data_compensation.compensation = 1
data_normal.compensation = 0

data = pd.concat([data_compensation, data_normal], axis=0) """

data_compensation_original.compensation = 1
data_normal_original.compensation = 0

data = pd.concat([data_compensation_original, data_normal_original], axis=0)
data[data.frame == 1]

,path,frame,compensation,x_0,y_0,z_0,x_1,y_1,z_1,x_2,...,z_29,x_30,y_30,z_30,x_31,y_31,z_31,x_32,y_32,z_32
0,data/video/compensation\01_trinken_kompensatio...,1,1,1000.09450,295.08030,-434.73780,1016.18540,265.61652,-381.51593,1025.41260,...,-142.551970,952.20350,1076.1581,-163.268000,1083.44380,1195.5463,-371.589500,882.91846,1209.9233,-408.508760
239,data/video/compensation\02_trinken_kompensatio...,1,1,1004.91460,272.73907,-510.62988,1019.11080,241.79927,-459.76190,1028.63460,...,-329.010470,894.49110,1262.0361,-357.484380,1198.84160,1341.8446,-638.522160,827.46140,1330.2144,-690.302400
529,data/video/compensation\03_trinken_kompensatio...,1,1,1020.42883,267.42682,-428.96457,1037.81420,237.42938,-372.71088,1048.06870,...,-39.149357,917.36487,1188.9327,-127.484695,1046.44620,1207.4360,-247.560130,865.76776,1265.4349,-387.392430
828,data/video/compensation\04_trinken_kompensatio...,1,1,1028.70000,259.41913,-509.90570,1046.98300,230.39180,-457.48190,1057.37130,...,171.714160,920.50530,1136.3898,-14.946421,1048.78490,1185.5167,-0.154483,862.41925,1285.5558,-266.175840
1174,data/video/compensation\05_trinken_kompensatio...,1,1,1026.52210,290.28644,-331.21103,1041.60880,261.87927,-267.49520,1051.12600,...,-132.645080,915.38214,1191.8250,-372.592470,992.26080,1183.2690,-359.144100,862.46747,1262.1813,-689.382500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62283,data/video/normal\Video77_trinken.mp4,1,0,667.29870,193.52133,-552.82400,688.31980,162.64233,-520.66880,700.90670,...,369.974700,608.46210,1256.4636,266.583830,760.96735,1309.6775,114.380890,629.08010,1314.1165,-3.877905
62528,data/video/normal\Video78_trinken.mp4,1,0,652.67970,189.19930,-445.65800,675.32500,158.94757,-412.61328,689.13110,...,23.497173,613.40380,1194.0247,-131.681350,772.03827,1252.9551,-252.936690,622.57104,1281.3988,-424.426880
62718,data/video/normal\Video7_trinken.mp4,1,0,738.33685,175.25648,-745.72520,760.37463,137.06886,-704.40000,772.15550,...,-192.227250,695.72754,903.2492,-127.926926,850.12690,967.2632,-399.477420,690.68616,960.9032,-337.431500
62827,data/video/normal\Video8_trinken.mp4,1,0,721.06476,162.20511,-909.26930,746.28710,128.84732,-867.53910,760.18646,...,576.729500,641.96985,1360.2854,693.434600,826.81555,1420.6348,210.978330,675.01105,1422.2444,302.922270


## 3.1 Data Centering

In [320]:
""" 
Zentrierung Datenpunkte für Datenanalyse 
Dies ist die Zentrierung des Avatars am Kopf, indem dieser zum Punkt (0,0,0) gebracht wird. 
Alle anderen Keypoints werden entsprechend diesen X, Y und Z Werten verschoben. 

Die Zentrierung wird anstelle des von Nils erstellten Ansatzes verwendet, 
damit die Daten für die Datenanalyse im späteren Schritt verwendet werden können.

"""
x_cols = []
y_cols = []
z_cols = []
for i in range(33):
    x_cols.append(f'x_{i}')
    y_cols.append(f'y_{i}')
    z_cols.append(f'z_{i}')


In [321]:
videos_raw = list(data.groupby(data.path))
len(videos_raw)

599

In [322]:
def center_keypoints(uncentered_videos):

    centred_df = pd.DataFrame()
    for path, video in uncentered_videos:
        keypoints = video[x_cols + y_cols + z_cols]

        # Step 2: get the head position and use it as cetner of mass
        center_of_mass = keypoints[['x_0', 'y_0', 'z_0']].iloc[0].values
        #print(center_of_mass)

        # Step 3: Translate keypoints
        tmp = pd.DataFrame()
        tmp[x_cols] = keypoints[x_cols] - center_of_mass[0]
        tmp[y_cols] = keypoints[y_cols] - center_of_mass[1]
        tmp[z_cols] = keypoints[z_cols] - center_of_mass[2]
        tmp.insert(0, 'path', path)
        tmp.insert(0, 'frame', video['frame'])
        tmp.insert(0, 'compensation', video['compensation'])
  
        centred_df = pd.concat([centred_df, tmp], axis=0, ignore_index=True)

    return centred_df
df_centred = center_keypoints(videos_raw)

/var/folders/y8/blsng9yn0_jdwkc6k9fx0f240000gn/T/ipykernel_75029/972185764.py:17: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  tmp.insert(0, 'frame', video['frame'])
/var/folders/y8/blsng9yn0_jdwkc6k9fx0f240000gn/T/ipykernel_75029/972185764.py:18: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  tmp.insert(0, 'compensation', video['compensation'])
/var/folders/y8/blsng9yn0_jdwkc6k9fx0f240000gn/T/ipykernel_75029/972185764.py:17: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert

In [323]:
df_centred.head(5)

,compensation,frame,path,x_0,x_1,x_2,x_3,x_4,x_5,x_6,...,z_23,z_24,z_25,z_26,z_27,z_28,z_29,z_30,z_31,z_32
0,1,1,data/video/compensation\01_trinken_kompensatio...,0.00000,16.09090,25.3181,34.5117,-14.88010,-28.05763,-40.49510,...,450.567650,419.513667,-315.66595,-372.67803,244.51726,221.62542,292.18583,271.46980,63.14830,26.22904
1,1,2,data/video/compensation\01_trinken_kompensatio...,-0.94710,15.09794,24.6263,33.5348,-17.40016,-29.90035,-41.75594,...,449.760284,420.319871,-435.47814,-472.88805,196.40887,210.35300,254.04658,269.09470,7.93418,20.98980
2,1,3,data/video/compensation\01_trinken_kompensatio...,-1.12496,14.84105,24.4170,33.1018,-18.16160,-30.35110,-42.03050,...,448.142590,421.979848,-462.23846,-463.09610,144.22202,187.04430,199.76705,245.86169,-56.07543,-6.63520
3,1,4,data/video/compensation\01_trinken_kompensatio...,-1.12850,14.83450,24.4029,33.0749,-18.25026,-30.36650,-42.03017,...,446.584930,423.564514,-441.82940,-447.72405,159.24262,171.48038,213.89739,227.15159,-36.82670,-29.00290
4,1,5,data/video/compensation\01_trinken_kompensatio...,-1.05160,15.00960,24.4962,33.2365,-18.23274,-30.27614,-41.85176,...,446.199955,423.970270,-440.79430,-467.01305,163.87144,137.69517,218.75971,191.70960,-28.91256,-71.34997


## 3.2 Export centered Data for Data Analysis

### 3.2.1 Export Datenset "Kompensiert"

In [324]:
df_centred_augmented = df_centred[df_centred.path.str.startswith('/Users/')]
df_centred_original = df_centred[df_centred.path.str.startswith('data/')]

df_centred_augmented_compensated = df_centred_augmented[df_centred_augmented['compensation'] == 1]
df_centred_augmented_compensated.head(5)

,compensation,frame,path,x_0,x_1,x_2,x_3,x_4,x_5,x_6,...,z_23,z_24,z_25,z_26,z_27,z_28,z_29,z_30,z_31,z_32


In [325]:
df_centred_augmented_compensated.to_csv('../Data/KeypointsBereinigtKompensiert.csv',
                           sep=",",
                           encoding='utf-8',
                           index=False) 

### 3.2.2 Export Datenset "Nicht Kompensiert"

In [326]:
df_centred_augmented_notCompensated = df_centred_augmented[df_centred_augmented['compensation'] == 0]
df_centred_augmented_notCompensated.head(5)

,compensation,frame,path,x_0,x_1,x_2,x_3,x_4,x_5,x_6,...,z_23,z_24,z_25,z_26,z_27,z_28,z_29,z_30,z_31,z_32


In [327]:
df_centred_augmented_notCompensated.to_csv('../Data/KeypointsBereinigtNichtKompensiert.csv',
                           sep=",",
                           encoding='utf-8',
                           index=False) 

In [328]:
""" df_centred_augmented = df_centred_augmented[['path', 'frame' ,'compensation', 'x_0', 'y_0', 'z_0', 'x_11', 'y_11', 'z_11',
           'x_12', 'y_12', 'z_12', 'x_13', 'y_13', 'z_13', 'x_14', 'y_14', 'z_14', 'x_15', 'y_15', 'z_15',
           'x_16', 'y_16', 'z_16', 'x_17', 'y_17', 'z_17', 'x_18', 'y_18', 'z_18', 'x_19', 'y_19', 'z_19',
           'x_20', 'y_20', 'z_20', 'x_21', 'y_21', 'z_21', 'x_22', 'y_22', 'z_22', 'x_23', 'y_23', 'z_23',
           'x_24', 'y_24', 'z_24', 'x_25', 'y_25', 'z_25', 'x_26', 'y_26', 'z_26', 'x_27', 'y_27', 'z_27',
           'x_28', 'y_28', 'z_28', 'x_31', 'y_31', 'z_31', 'x_32', 'y_32', 'z_32']] 

df_centred_original = df_centred_original[['path', 'frame' ,'compensation', 'x_0', 'y_0', 'z_0', 'x_11', 'y_11', 'z_11',
           'x_12', 'y_12', 'z_12', 'x_13', 'y_13', 'z_13', 'x_14', 'y_14', 'z_14', 'x_15', 'y_15', 'z_15',
           'x_16', 'y_16', 'z_16', 'x_17', 'y_17', 'z_17', 'x_18', 'y_18', 'z_18', 'x_19', 'y_19', 'z_19',
           'x_20', 'y_20', 'z_20', 'x_21', 'y_21', 'z_21', 'x_22', 'y_22', 'z_22', 'x_23', 'y_23', 'z_23',
           'x_24', 'y_24', 'z_24', 'x_25', 'y_25', 'z_25', 'x_26', 'y_26', 'z_26', 'x_27', 'y_27', 'z_27',
           'x_28', 'y_28', 'z_28', 'x_31', 'y_31', 'z_31', 'x_32', 'y_32', 'z_32']] """

" df_centred_augmented = df_centred_augmented[['path', 'frame' ,'compensation', 'x_0', 'y_0', 'z_0', 'x_11', 'y_11', 'z_11',\n           'x_12', 'y_12', 'z_12', 'x_13', 'y_13', 'z_13', 'x_14', 'y_14', 'z_14', 'x_15', 'y_15', 'z_15',\n           'x_16', 'y_16', 'z_16', 'x_17', 'y_17', 'z_17', 'x_18', 'y_18', 'z_18', 'x_19', 'y_19', 'z_19',\n           'x_20', 'y_20', 'z_20', 'x_21', 'y_21', 'z_21', 'x_22', 'y_22', 'z_22', 'x_23', 'y_23', 'z_23',\n           'x_24', 'y_24', 'z_24', 'x_25', 'y_25', 'z_25', 'x_26', 'y_26', 'z_26', 'x_27', 'y_27', 'z_27',\n           'x_28', 'y_28', 'z_28', 'x_31', 'y_31', 'z_31', 'x_32', 'y_32', 'z_32']] \n\ndf_centred_original = df_centred_original[['path', 'frame' ,'compensation', 'x_0', 'y_0', 'z_0', 'x_11', 'y_11', 'z_11',\n           'x_12', 'y_12', 'z_12', 'x_13', 'y_13', 'z_13', 'x_14', 'y_14', 'z_14', 'x_15', 'y_15', 'z_15',\n           'x_16', 'y_16', 'z_16', 'x_17', 'y_17', 'z_17', 'x_18', 'y_18', 'z_18', 'x_19', 'y_19', 'z_19',\n           'x_20

In [329]:
# Videos Original gruppieren und in Liste schreiben
videos_raw_augmented = list(df_centred_augmented.groupby(df_centred_augmented.path))
len(videos_raw_augmented)

# Videos Augmentiert gruppieren und in Liste schreiben
videos_raw_original = list(df_centred_original.groupby(df_centred_original.path))
len(videos_raw_original)

599

## 3.3 Pre-Processing-Funktionen

In [330]:
'''
Diese Funktion bringt die rohen Videos in eine neue Form, ohne die Videos zu schneiden.
Sie wird für den Sliding Window Ansatz benötigt.
Aus den rohen Videos werden nur noch die exrahierten Datenpunkte als Liste zurückgegeben.
'''
def remap_raw_videos(unmapped_videos):
    remapped_videos = []
    for video in unmapped_videos:
        v = video[1].reset_index()
        mapped_vid = v.loc[:, 'x_0':]
        remapped_videos.append(mapped_vid)
    return remapped_videos

In [331]:
'''
Schneiden der Videos

Diese Funktion bringt die rohen Videos in eine neue Form und schneidet sie eine bestimmte Anzahl Frames
vor und hinter dem vertikalen Höhepunkt der trinkenden Hand ab.

Aus den rohen Videos werden nur noch die exrahierten Datenpunkte als Liste zurückgegeben.

Inputs:
uncutted_videos: Die ungeschnittenen rohen Videos
n_frames: Die Anzahl der Frames, welche vor und nach dem Höchstpunkt der Hand (y_16) abgeschnitten werden sollen
'''
def cut_videos(uncutted_videos, n_frames=0):
    cut_videos = []
    for idx, video in enumerate(uncutted_videos):
        v = video[1].reset_index()
        # Position des höchsten Punktes (tiefster Wert, da tiefere Werte höhere vertikale Positionen aussagen)
        minpos = np.argmin(v.y_16)
        
        # Falls der Höhepunkt der Hand zu nahe am Beginn / Ende des Videos ist, ist die Trinkbewegung vermutlich nicht korrekt.
        # Das Video soll dann optimalerweise gelöscht werden.
        if((minpos < 20) or (len(v) - minpos < 20)):
            print('\033[91m' + 'Video mit dem Namen\n'
                  + videos_raw[idx][0]
                  + '\nund Index\n' + str(idx)
                  + '\nzeigt keine korrekte Trinkbewegung. Bitte entfernen.')
            
        cut_vid = v.loc[minpos-n_frames:minpos+n_frames, 'x_0':]   # Video x Frames vor und x Frames nach Höhepunkt der Hand (y_16) abschneiden
        cut_videos.append(cut_vid)
    return cut_videos

In [332]:
'''
Videos zentrieren

Diese Funktion zentriert alle Videos, in dem von allen Landmarks die Position des Kopfes im ersten Frame abgezogen wird.
'''
def center_data(uncentered_videos):
    centered_videos = []
    for video in uncentered_videos:
        centered_video = []
        # Position des Kopfes im ersten Frame des Videos bestimmen (x, y und z-Koordinate)
        head_start = video.loc[:,'x_0':'z_0'].values[0]
        for frame in np.array(video):
            centered_frame = []
            # Frame reshapen, sodass alle Landmarks als eine Liste zählen
            # Ohne Nullpunkte: 21
            landmarks = frame.reshape((21, 3))
            for landmark in landmarks:
                centered_frame.append(landmark - head_start)
            centered_video.append(list(np.array(centered_frame).flatten()))
        centered_videos.append(centered_video)
    return centered_videos

In [333]:
'''
Relative Abstände

Diese Funktion berechnet den Abstand jedes Punktes des Skeletts zum Kopf und gibt diese anschliessend zurück.

Shape der Rückgabe: [x Anzahl Videos, x Anzahl Frames, 33 Datenpunkte]
'''
def calc_distances(raw_videos):
    # Abstand zu Kopf
    distances = []

    for video in raw_videos:
        frame_distances = []
        for frame in np.array(video):
            points = frame.reshape((33, 3))
            point_distances = []
            for k in range(len(points)):
                # Distanz einzeln von x-, y- und z-Koordinaten
                distance = points[k]-points[0]
                # Distanz mittels Formel berechnen
                point_distances.append(math.sqrt(distance[0] ** 2 + distance[1] ** 2 + distance[2] ** 2))
            frame_distances.append(point_distances)
        distances.append(frame_distances)
    return distances

In [334]:
'''
Diese Funktion gibt die Labels der Videos zurück.
1 = Compensation
0 = Natural
'''
def define_labels():
    labels = []

    for i in range(len(videos_raw_original)):
        labels.append(np.mean(videos_raw_original[i][1].compensation))
        
    for i in range(len(videos_raw_augmented)):
        labels.append(np.mean(videos_raw_augmented[i][1].compensation))
        
    return labels
labels = define_labels()

In [335]:
'''
Sliding Windows

Diese Funktion erstellt für alle Videos von unslided_videos Sliding Windows und gibt sie zurück.
Ausserdem werden die Labels auf die Sliding Windows korrekt verteilt.

Shape der Rückgabe: [x Anzahl Sliding Windows, {window_size} Anzahl Frames, 99 Datenpunkte]
'''
def create_sliding_windows(unslided_videos, window_size):
    videos_slided = []
    unslided_labels = define_labels()
    slided_labels = []
    for idx, unslided_video in enumerate(unslided_videos):
        video_label = unslided_labels[idx]
        for i in range(len(unslided_video) - window_size + 1):
            videos_slided.append(unslided_video[i:i+window_size])
            slided_labels.append(video_label)
    return videos_slided, slided_labels

In [336]:
'''
In dieser Funktion werden die Daten in eine geeignete Form gebracht.
Es werden die Values der einzelnen Videos in eine Liste geschrieben und zurückgegeben.
'''
def reshape_videos(unshaped_videos):
    reshaped_videos = []
    labels = []
    for video in unshaped_videos:
        reshaped_videos.append(video.values)
    return reshaped_videos

## 3.4 Ansatz auswählen ✏️

In [337]:
# Ansatz 1: Absolute Datenpunkte mit geschnittenen Videos
# videos = reshape_videos(cut_videos(videos_raw, n_frames=38))

# Ansatz 2: Zentrierte Datenpunkte mit geschnittenen Videos
# videos = center_data(cut_videos(videos_raw, n_frames=10))

# Ansatz 3: Relative Datenpunkte (Abstände zum Kopf) mit geschnittenen Videos
# videos = calc_distances(cut_videos(videos_raw, n_frames=10))

# Ansatz 4a: Sliding Windows mit absoluten Datenpunkten
videos_original, labels_original = create_sliding_windows(remap_raw_videos(videos_raw_original), 5)
videos_augmented, labels_augmented = create_sliding_windows(remap_raw_videos(videos_raw_augmented), 5)

# Ansatz 4b: Sliding Windows mit zentrierten Datenpunkten
# videos, labels = create_sliding_windows(center_data(remap_raw_videos(videos_raw)), 10)

# Ansatz 4b: Sliding Windows mit relativen Datenpunkten
# videos, labels = create_sliding_windows(calc_distances(remap_raw_videos(videos_raw)), 10)

np.array(videos_original).shape
np.array(videos_augmented).shape

(0,)

In [338]:
'''
Falls es zu einem Fehler kommt, kann folgende Zeile angepasst und auskommentiert werden, um ein falsches Video zu entfernen.
Alternativ kann das Video im Ordner gelöscht werden und die Datenpunkte neu extrahiert werden. ✏️
'''
# videos_raw.remove(videos_raw[144])

'\nFalls es zu einem Fehler kommt, kann folgende Zeile angepasst und auskommentiert werden, um ein falsches Video zu entfernen.\nAlternativ kann das Video im Ordner gelöscht werden und die Datenpunkte neu extrahiert werden. ✏️\n'

## 3.5 Features und Labels definieren

In [339]:
# Features bestimmen
X = np.asarray(videos_original)
X.shape

(145743, 5, 99)

In [340]:
# Labels setzen
y = np.array(labels_original)

y.shape

(145743,)

In [341]:
# Features bestimmen
X_train_salome = np.asarray(videos_augmented)
X_train_salome.shape

(0,)

In [342]:
# Labels setzen
y_train_salome = np.array(labels_augmented)

y_train_salome.shape

(0,)

In [343]:
# Daten in ein Trainings- und Testset unterteilen
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=4)

In [344]:
""" 
Augmentierte Daten werden dem Trainingsset hinzugefügt 

# Concatenate the additional data with the existing X_train and y_train
X_train = np.concatenate((X_train, X_train_salome), axis=0)
y_train = np.concatenate((y_train, y_train_salome), axis=0)

# Ensure the shape of X_train and y_train matches after concatenation
assert X_train.shape[0] == y_train.shape[0]

# Optionally, shuffle the concatenated data
indices = np.arange(X_train.shape[0])
np.random.shuffle(indices)
X_train = X_train[indices]
y_train = y_train[indices] 

"""

' \nAugmentierte Daten werden dem Trainingsset hinzugefügt \n\n# Concatenate the additional data with the existing X_train and y_train\nX_train = np.concatenate((X_train, X_train_salome), axis=0)\ny_train = np.concatenate((y_train, y_train_salome), axis=0)\n\n# Ensure the shape of X_train and y_train matches after concatenation\nassert X_train.shape[0] == y_train.shape[0]\n\n# Optionally, shuffle the concatenated data\nindices = np.arange(X_train.shape[0])\nnp.random.shuffle(indices)\nX_train = X_train[indices]\ny_train = y_train[indices] \n\n'

In [345]:
# Daten in ein Trainings- und Testset unterteilen
X_validate, X_test, y_validate, y_test = train_test_split(X_test, y_test, test_size=0.5, stratify=y_test, random_state=2)

In [346]:
# Verteilung der Testdaten
np.mean(y_test)

0.5756563900832494

# 4. Modelle trainieren

## 4.1 LSTM

In [347]:
def define_lstm_model(input_shape, n_neurons=1):
    lstm_model = keras.Sequential()
    lstm_model.add(layers.LSTM(n_neurons, activation='relu', input_shape=input_shape))
    lstm_model.add(layers.Dense(1, activation='sigmoid'))
    return lstm_model

## 4.2 RNN

In [348]:
def define_rnn_model(input_shape, n_neurons=1):
    rnn_model = keras.Sequential()
    rnn_model.add(layers.SimpleRNN(n_neurons, activation='relu', input_shape=input_shape))
    rnn_model.add(layers.Dense(1, activation='sigmoid'))
    return rnn_model

## 4.3 Modell anwenden

In [349]:
'''
Gewünschtes Modell erstellen

Auskommentieren und Anzahl Neuronen anpassen (n_neurons)
input_shape wird automatisch übernommen
'''
# Anzahl Neuronen bestimmen ✏️
n_neurons=64

# LSTM
# model = define_lstm_model(n_neurons=n_neurons, input_shape=X.shape[1:])

# RNN
model = define_rnn_model(n_neurons=n_neurons, input_shape=X.shape[1:])

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [350]:
# Modell kompilieren
model.compile(
    loss='binary_crossentropy',
    optimizer="Adam",
    metrics=['accuracy'],
)

In [351]:
X_train.shape

(102020, 5, 99)

In [352]:
# Modell trainieren
model.fit(
    X_train, y_train, validation_data=(X_validate, y_validate), epochs=1000
)
# validation_data=(X_validate, y_validate),

Epoch 1/1000
3189/3189 ━━━━━━━━━━━━━━━━━━━━ 3s 822us/step - accuracy: 0.8512 - loss: 8.0902 - val_accuracy: 0.9537 - val_loss: 0.6854
Epoch 2/1000
3189/3189 ━━━━━━━━━━━━━━━━━━━━ 2s 751us/step - accuracy: 0.9351 - loss: 1.2209 - val_accuracy: 0.9467 - val_loss: 0.7353
Epoch 3/1000
3189/3189 ━━━━━━━━━━━━━━━━━━━━ 2s 730us/step - accuracy: 0.9520 - loss: 0.6166 - val_accuracy: 0.9387 - val_loss: 0.6667
Epoch 4/1000
3189/3189 ━━━━━━━━━━━━━━━━━━━━ 2s 711us/step - accuracy: 0.9612 - loss: 0.3374 - val_accuracy: 0.9568 - val_loss: 0.1766
Epoch 5/1000
3189/3189 ━━━━━━━━━━━━━━━━━━━━ 2s 701us/step - accuracy: 0.9693 - loss: 0.1289 - val_accuracy: 0.9788 - val_loss: 0.0592
Epoch 6/1000
3189/3189 ━━━━━━━━━━━━━━━━━━━━ 2s 698us/step - accuracy: 0.9737 - loss: 0.0771 - val_accuracy: 0.9640 - val_loss: 0.1087
Epoch 7/1000
3189/3189 ━━━━━━━━━━━━━━━━━━━━ 2s 727us/step - accuracy: 0.9762 - loss: 0.0704 - val_accuracy: 0.9850 - val_loss: 0.0440
Epoch 8/1000
3189/3189 ━━━━━━━━━━━━━━━━━━━━ 2s 733us/step - ac

In [353]:
model.summary()

Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ simple_rnn_5 (SimpleRNN)        │ (None, 64)             │        10,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 31,685 (123.77 KB)

 Trainable params: 10,561 (41.25 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 21,124 (82.52 KB)

### Modell speichern ✏️

In [354]:
# model_name = 'models/LSTM/lstm_a_cv_'
# model_name = 'models/RNN/rnn_a_cv_'
# model_name = 'models/sliding_window/sw_lstm_a_cv_'
model_name = 'models/sliding_window/sw_rnn_a_cv_'

# Wenn das Modell gespeichert werden soll, folgende Zeile auskommentieren:
# LEGACY
# model.save(model_name + str(X.shape[1]) + 'f_' + str(n_neurons) + 'n/' + 'model.h5')

model.save(model_name + str(X.shape[1]) + 'f_' + str(n_neurons) + 'n/' + 'my_model.keras')

## 4.4 Hyperparametertuning

In [355]:
'''
Hyperparametertuning

Diese Funktion findet die Anzahl an Neuronen, welche das beste Resultat liefert.
'''
def tune_hyperparameters(untuned_model):
    # Regressor mit dem mitgegebenen Modell erstellen
    keras_regressor = keras.wrappers.scikit_learn.KerasRegressor(untuned_model)

    # Eine Suche mit dem Regressor erstellen. Es werden Werte der Neuronen zwischen 1 und 128 ausgewertet.
    randomized_search_cv = RandomizedSearchCV(
        keras_regressor,
        {"n_neurons": np.arange(1, 128)},
        n_iter=10,
        cv=3
    )
    
    randomized_search_cv.fit(X_train, y_train, validation_data=(X_validate, y_validate), epochs=1000)
    
    print(randomized_search_cv.best_params_)
    
    return randomized_search_cv.best_estimator_.model

In [356]:
# Auskommentieren und Modell anpassen, um das beste Modell zu finden und zu speichern ✏️
# best_model = tune_hyperparameters(untuned_model=define_lstm_model)
# best_model.save('models/best_model')

# 5. Predictions

In [357]:
# Auskommentieren und ein bestimmtes Modell zu laden ✏️
# model = keras.models.load_model('models/RNN/rnn_a_cv_21f_32n')

In [358]:
result = model.predict(X_test)

result

684/684 ━━━━━━━━━━━━━━━━━━━━ 0s 409us/step


array([[1.       ],
       [1.       ],
       [1.       ],
       ...,
       [1.       ],
       [0.9995811],
       [1.       ]], dtype=float32)

# 6. Evaluation

In [359]:
scores = model.evaluate(X_train, y_train, verbose=0)
print("In-Sample Accuracy: %.2f%%" % (scores[1]*100))
scores = model.evaluate(X_test, y_test, verbose=0)
print("Out-of-Sample Accuracy: %.2f%%" % (scores[1]*100))

In-Sample Accuracy: 99.63%
Out-of-Sample Accuracy: 99.52%


In [360]:
result = (result > 0.5).flatten()
actual = y_test > 0.5

In [361]:
confusion_matrix(actual, result)

array([[ 9239,    38],
       [   68, 12517]])

## Evaluation eines einzelnen Videos

In [362]:
# Gewünschtes Sliding-Window-Modell auskommentieren ✏️

model = keras.models.load_model('models/sliding_window/sw_rnn_a_cv_5f_64n/my_model.keras')
# model = keras.models.load_model('models/sliding_window/sw_lstm_a_cv_5f_64n')

# model = keras.layers.TFSMLayer('models/sliding_window/sw_rnn_a_cv_10f_64n')
# model = keras.models.load_model('models/sliding_window/sw_rnn_a_cv_77f_64n/my_model.keras')
# model = keras.models.load_model('models/sliding_window/sw_lstm_a_cv_10f_64n')
# model = keras.models.load_model('models/sliding_window/sw_rnn_a_cv_20f_64n')
# model = keras.models.load_model('models/sliding_window/sw_lstm_a_cv_20f_64n')

In [363]:
'''
Video klassifizieren

Diese Funktion berechnet für ein Video die Kompensationswahrscheinlichkeit

Input:
video_index: Index des Videos
window_size: Grösse der Sliding Windows

Output: Liste mit Wahrscheinlichkeiten für alle Sliding Windows der Videos
'''
def predict_video(video_index, window_size=5):
    return model.predict(np.array(create_sliding_windows(remap_raw_videos([videos_raw[video_index]]), window_size)[0]))